In [ ]:
# Dependencis I
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import pandas as pd
import datetime as dt
plt.style.use('dark_background')
x = np.linspace(0, 10)

In [ ]:
# Depencies II
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
#SQL Alchemy Connection
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

#  Tables Classes 

In [ ]:
# Modify Existing Database 
Base = automap_base()

# Classify tables
Base.prepare(engine, reflect=True)

In [ ]:
# Table views generated by automap 
Base.classes.keys()

In [ ]:
# Transform ref for table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Session Engine
session = Session(engine)

# Precipitation Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data.


end_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
last_date = end_date.date

# Convert last_date from string to date
last_date = dt.datetime.strptime(last_date, "%Y-%m-%d").date()

query_date = last_date - dt.timedelta(days=365)

# Query precipitation values
last_year_prcp = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= query_date).all()

# Load the query results as Pandas DataFrame and set the index to the date column
prcp_df = pd.DataFrame(last_year_prcp).set_index('date',drop=True).dropna(how='any')

# Sort the dataframe by date
prcp_sorted_df = prcp_df.sort_index()

# Use Pandas Plotting with Matplotlib to plot the data
prcp_sorted_df.plot(figsize=(10,6))

plt.xlabel('Date')
plt.ylabel('Precipitaion Values')
plt.xticks(Rotation = 45)
plt.legend(['Annual Precipitation'])

plt.savefig("Images/prcp_12months.png")
plt.show()

In [ ]:
# calculate the summary statistics 
summary_stats = prcp_sorted_df.describe()
summary_stats

### Station Analysis

In [ ]:
# Query the count of weather stations 
stations_count = session.query(Station.station).count()
stations_count

In [ ]:
# Most active stations in ascending order
most_active_stations = session.query(Measurement.station, func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
most_active_stations

In [ ]:
# station with most observations.
most_active_station = most_active_stations[0][0]

print(f'Station with MOST observations is {most_active_station}.')

In [ ]:
# Calculate 
#the lowest temperature recorded, 
# highest temperature recorded, and 
# ave temperature of Station with MOST observations
Temp_Stats = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).filter(Measurement.station == most_active_station).all()
lowest_temp = Temp_Stats[0][0]
highest_temp =Temp_Stats[0][1]
avg_temp = round(Temp_Stats[0][2],2)

print(f'{most_active_station} has a min temp of {lowest_temp} F.')
print(f'{most_active_station} has a max temp of {highest_temp} F.')
print(f'{most_active_station} has a avg temp of {avg_temp} F.')


In [ ]:
#* Filter by the station with the highest number of observations.
#* Plot the results as a histogram with `bins=12`.

Annual_Max_tobs = session.query(Measurement.tobs).filter(Measurement.station == most_active_station, Measurement.date >= query_date).all()

# Save the query results as a Pandas DataFrame
tobs_df = pd.DataFrame(Annual_Max_tobs)
tobs_df

# Plot results on a histogram
tobs_df.plot(kind='hist', bins=12, figsize=(10,6))
plt.xlabel('Temperature (f)')
plt.ylabel('Observations')
plt.legend(['Annual Observations'])

plt.savefig("Images/tobs_12months.png")
plt.show()

## Bonus - Temperature Analysis II

In [ ]:
# Create function `Stat_Analysis` to make calculations. 

def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d  
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()




In [ ]:
# Use function `calc_temps` to calculate  tmin, tavg, and tmax 

vacay_start = '2017-04-25'
vacay_end = '2017-05-15'

# Convert trip stat/end dates fron string to date
vacay_yr_start = ((dt.datetime.strptime(vacay_start, "%Y-%m-%d").date()) - dt.timedelta(days=365)).strftime('%Y-%m-%d')
vacay_yr_end = ((dt.datetime.strptime(vacay_end, "%Y-%m-%d")) - dt.timedelta(days=365)).strftime('%Y-%m-%d')

vacay_temps=(calc_temps(vacay_yr_start, vacay_yr_end))
vacay_temps

In [ ]:
#Plot the min, avg, and max temperature from previous query as a bar chart.
#Use the average temperature as the bar height.
#Use the peak-to-peak (TMAX-TMIN) value as the y error bar (YERR).

tmax = vacay_temps[0][2]
tmin = vacay_temps[0][0]
tavg = vacay_temps[0][1]

plt.figure(figsize=(2.5,8))
plt.bar(x=1, height=tavg, yerr=(tmax-tmin), alpha=0.7, color='Teal');
plt.ylim(0,105)
plt.title("Vacay Ave Temp")
plt.ylabel("Temp (F)")
plt.xticks([])

plt.tight_layout()
plt.savefig("Images/Vacay_ave_temp.png")
plt.show()